# A Guided Tour of Ray Core: Remote Classes

[*Remote Classes*](https://docs.ray.io/en/latest/walkthrough.html#remote-classes-actors)
involve using a `@ray.remote` decorator on a class. 

This implements an [*actor*](https://patterns.eecs.berkeley.edu/?page_id=258) pattern, with properties: *stateful*, *message-passing semantics*

---

First, let's start Ray…

In [28]:
import logging
import ray
from random import randint

In [29]:
ray.init(
    ignore_reinit_error=True,
    logging_level=logging.ERROR,
)

{'node_ip_address': '127.0.0.1',
 'raylet_ip_address': '127.0.0.1',
 'redis_address': '127.0.0.1:6379',
 'object_store_address': '/tmp/ray/session_2021-12-23_15-12-27_315052_25779/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2021-12-23_15-12-27_315052_25779/sockets/raylet',
 'webui_url': '127.0.0.1:8265',
 'session_dir': '/tmp/ray/session_2021-12-23_15-12-27_315052_25779',
 'metrics_export_port': 64131,
 'node_id': '2b5f9344b8c379460e29d343e26d0cda787371ddffe972f9e2e52287'}

## 3. Remote Class as a Stateful Actor Pattern

To start, we'll define a class and use the decorator:

Let's use Python class and convert that to a remote Actor class and create multiple actor handle instances associated with a distinct attributes, such as a name, age, goals scored, etc

In [30]:
@ray.remote
class GoalsScored:
    def __init__ (self, player, age) -> None:
        self._goals = 0
        self._player = player
        self._age = age

    def score (self, goal) -> object:
        self._goals += goal
        return self._goals
       
    def player(self) -> str:
        return self._player
    
    # Any method of the actor can return multiple object refs.
    @ray.method(num_returns=3)
    def stats(self) -> object:
        return self._player, self._age, self._goals

Define three Actors: Rolando, Neymar, Messi

In [31]:
%%time 

ronaldo = GoalsScored.remote("Ronaldo", randint(18, 35))
neymar = GoalsScored.remote("Neymar", randint(18, 35))
messi = GoalsScored.remote("Messi", randint(18, 35))

CPU times: user 7.64 ms, sys: 2.66 ms, total: 10.3 ms
Wall time: 8.42 ms


Update the scores for each player

In [32]:
%%time

ronaldo_ref = ronaldo.score.remote(randint(1, 7))
neymar_ref = neymar.score.remote(randint(1, 7))
messi_ref  = messi.score.remote(randint(1, 7))

CPU times: user 1.41 ms, sys: 1.2 ms, total: 2.61 ms
Wall time: 1.03 ms


Again, use list comprehension to iterate over each Actor handle instances, along with object_ref for their goals scores, maintained by each distinct actor.

In [33]:
for ref, ref_obj in [(ronaldo, ronaldo_ref), (neymar, neymar_ref), (messi, messi_ref) ]:
   print(f"Player: {ray.get(ref.stats.remote())} and goals scored: {ray.get(ref_obj)}")

Player: ['Ronaldo', 33, 3] and goals scored: 3
Player: ['Neymar', 27, 7] and goals scored: 7
Player: ['Messi', 28, 6] and goals scored: 6


Finally, shutdown Ray

In [34]:
ray.shutdown()

---
## References

 * [A Universal Modular Actor Formalism for Artificial Intelligence](https://www.ijcai.org/Proceedings/73/Papers/027B.pdf)  
Carl Hewitt, Peter Bishop, Richard Steiger  *IJCAI* (1973)
 * [Using and Programming with Actors](https://docs.ray.io/en/latest/actors.html)